# Data Preprocessing 


### Import Packages
Importing Numpy, Pandas, Tensorflow and Keras modules Preprocessing and Utils

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import preprocessing , utils, layers , activations , models
import pandas as pd

### Reading the data from the files
We parse each of the `.yaml` files.

*   Concatenate two or more sentences if the answer has two or more of them.
*   Remove unwanted data types which are produced while parsing the data.
*   Append `<START>` and `<END>` to all the `answers`.
*   Create a `Tokenizer` and load the whole vocabulary ( `questions` + `answers` ) into it.

In [2]:
from tensorflow.keras import preprocessing , utils
import os
import yaml

dir_path = './data'
files_list = os.listdir(dir_path + os.sep)

questions = list()
answers = list()

for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
        if len( con ) > 2 :
            questions.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append( ans )
        elif len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])

answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 1894


### C) Preparing data for Seq2Seq model

Our model requires three arrays namely `encoder_input_data`, `decoder_input_data` and `decoder_output_data`.

For `encoder_input_data` :
* Tokenize the `questions`. Pad them to their maximum length.

For `decoder_input_data` :
* Tokenize the `answers`. Pad them to their maximum length.

For `decoder_output_data` :

* Tokenize the `answers`. Remove the first element from all the `tokenized_answers`. This is the `<START>` element which we added earlier.

In [3]:
# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

# Saving all the arrays to storage
np.save( 'enc_in_data.npy' , encoder_input_data )
np.save( 'dec_in_data.npy' , decoder_input_data )
np.save( 'dec_tar_data.npy' , decoder_output_data )
np.save( 'vocab-size.npy' , VOCAB_SIZE )

(564, 22) 22
(564, 74) 74
(564, 74, 1894)


In [4]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))

encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 200)    378800      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 200)    378800      input_2[0][0]                    
______________________________________________________________________________________________

In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=40, epochs=200)
model.save( 'model.h5' ) 

Train on 564 samples
Epoch 1/200
564/564 [==============================] - 11s 20ms/sample - loss: 1.2684
Epoch 2/200
564/564 [==============================] - 4s 8ms/sample - loss: 1.1128
Epoch 3/200
564/564 [==============================] - 5s 8ms/sample - loss: 1.0877
Epoch 4/200
564/564 [==============================] - 5s 9ms/sample - loss: 1.0644
Epoch 5/200
564/564 [==============================] - 5s 8ms/sample - loss: 1.0444
Epoch 6/200
564/564 [==============================] - 5s 9ms/sample - loss: 1.0291
Epoch 7/200
564/564 [==============================] - 5s 8ms/sample - loss: 1.0137
Epoch 8/200
564/564 [==============================] - 5s 10ms/sample - loss: 0.9987
Epoch 9/200
564/564 [==============================] - 5s 9ms/sample - loss: 0.9833
Epoch 10/200
564/564 [==============================] - 6s 10ms/sample - loss: 0.9668
Epoch 11/200
564/564 [==============================] - 5s 10ms/sample - loss: 0.9483
Epoch 12/200
564/564 [==========================

564/564 [==============================] - 6s 11ms/sample - loss: 0.1532
Epoch 97/200
564/564 [==============================] - 6s 11ms/sample - loss: 0.1488
Epoch 98/200
564/564 [==============================] - 5s 8ms/sample - loss: 0.1447
Epoch 99/200
564/564 [==============================] - 5s 10ms/sample - loss: 0.1426
Epoch 100/200
564/564 [==============================] - 6s 11ms/sample - loss: 0.1376
Epoch 101/200
564/564 [==============================] - 5s 10ms/sample - loss: 0.1328
Epoch 102/200
564/564 [==============================] - 6s 10ms/sample - loss: 0.1295
Epoch 103/200
564/564 [==============================] - 5s 8ms/sample - loss: 0.1265
Epoch 104/200
564/564 [==============================] - 5s 9ms/sample - loss: 0.1233
Epoch 105/200
564/564 [==============================] - 6s 10ms/sample - loss: 0.1180
Epoch 106/200
564/564 [==============================] - 5s 10ms/sample - loss: 0.1151
Epoch 107/200
564/564 [==============================] - 5s 9ms

564/564 [==============================] - 5s 9ms/sample - loss: 0.0159
Epoch 191/200
564/564 [==============================] - 5s 8ms/sample - loss: 0.0151
Epoch 192/200
564/564 [==============================] - 5s 9ms/sample - loss: 0.0149
Epoch 193/200
564/564 [==============================] - 5s 8ms/sample - loss: 0.0152
Epoch 194/200
564/564 [==============================] - 5s 9ms/sample - loss: 0.0147
Epoch 195/200
564/564 [==============================] - 5s 9ms/sample - loss: 0.0146
Epoch 196/200
564/564 [==============================] - 5s 9ms/sample - loss: 0.0148
Epoch 197/200
564/564 [==============================] - 5s 9ms/sample - loss: 0.0143
Epoch 198/200
564/564 [==============================] - 5s 9ms/sample - loss: 0.0144
Epoch 199/200
564/564 [==============================] - 5s 9ms/sample - loss: 0.0144
Epoch 200/200
480/564 [========================>.....] - ETA: 0s - loss: 0.0139

In [ ]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model


def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

In [ ]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )